# Analyse edge cases of $SO(3)$ $\log$ map, derive Jacobians

In [ ]:
import sophus as sp
import numpy as np
import sys

## 1. Log maps

$$ \log(R)^{\vee} = \vec{v}$$ 
$R \in SO(3)$ - rotation matrix, $\vec{v} \in \mathbb{R}^3$ - angle/axis representation of rotation: $\vec{v} = \theta \vec{n}$, $\theta \in [0, \pi]$ - angle of rotation, $\vec{n}: ||{\vec{n}}||_2 = 1$ - unit vector corresponding to axis of rotation.

In [ ]:
class SO3():
    @staticmethod
    def log_pi(R):
        trR = R[0, 0] + R[1, 1] + R[2, 2]
        cos_theta = max(min(0.5 * (trR - 1), 1), -1)
        sin_theta = 0.5 * np.sqrt(max(0, (3 - trR) * (1 + trR)))
        theta = np.arctan2(sin_theta, cos_theta)
        R_minus_R_T_vee = np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])
        
        if abs(3 - trR) < 1e-8:
            # return log map at Theta = 0
            c = 0.5 * (1 + theta*theta / 6 + 7 / 360 * (theta**4))
            return c * R_minus_R_T_vee

        S = R + R.transpose() + (1 - trR) * np.eye(3)
        rest_tr = 3 - trR
        n = np.ones(3)
        # Fix modules of n_i
        for i in range(3):
            n[i] = np.sqrt(max(0, S[i, i] / rest_tr))
        max_i = np.argmax(n)
        
        # Fix signs according to the sign of max element
        for i in range(3):
            if i != max_i:
                n[i] *= np.sign(S[max_i, i])

        # Fix an overall sign
        if any(np.sign(n) * np.sign(R_minus_R_T_vee) < 0):
            n = -n
        return theta * n
    
    @staticmethod
    def log_baseline(R):    
        trR = R[0, 0] + R[1, 1] + R[2, 2]
        cos_theta = max(min(0.5 * (trR - 1), 1), -1)
        sin_theta = 0.5 * np.sqrt(max(0, (3 - trR) * (1 + trR)))
        theta = np.arctan2(sin_theta, cos_theta)
        R_minus_R_T_vee = np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])

        if abs(3 - trR) < 1e-8:
            # return log map at Theta = 0
            c = 0.5 * (1 + theta*theta / 6 + 7 / 360 * (theta**4))
            return c * R_minus_R_T_vee

        # it diverges around theta=pi
        v = theta / (2 * sin_theta) * R_minus_R_T_vee
        return v
    
    @staticmethod
    def matrix2quaternion(R):    
        # quaternion [q_0, q_{1:3}] = [c, vec]
        q = np.zeros(4)
        t = R[0, 0] + R[1, 1] + R[2, 2]
        if t > 0:
            # case 1
            t = np.sqrt(1 + t)
            q[0] = 0.5 * t
            t = 0.5 / t
            q[1] = (R[2, 1] - R[1, 2]) * t
            q[2] = (R[0, 2] - R[2, 0]) * t
            q[3] = (R[1, 0] - R[0, 1]) * t
        else:
            i = 0
            if R[1, 1] > R[0, 0]:
                i = 1
            if R[2, 2] > R[i, i]:
                i = 2
            j = (i + 1) % 3
            k = (j + 1) % 3
            t = np.sqrt(R[i, i] - R[j, j] - R[k, k] + 1)
            q[1+i] = 0.5 * t
            t = 0.5 / t
            q[0] = (R[k, j] - R[j, k]) * t
            q[1+j] = (R[j, i] + R[i, j]) * t
            q[1+k] = (R[k, i] + R[i, k]) * t
        return q
    
    @staticmethod
    def quaternion2vec(q):
        squared_n = q[1]*q[1] + q[2]*q[2] + q[3]*q[3]
        w = q[0]
        theta = 0
        if squared_n < np.finfo(float).eps * np.finfo(float).eps:
            squared_w = w*w
            two_atan_nbyw_by_n = 2 / w - 2.0 / 3 * squared_n / (w * squared_w) 
            theta = 2 * squared_n / w
        else:
            n = np.sqrt(squared_n)
            if (abs(w) < np.finfo(float).eps):
                if w >= 0:
                    two_atan_nbyw_by_n = np.pi / n
                else:
                    two_atan_nbyw_by_n = -np.pi / n
            else:
                two_atan_nbyw_by_n = 2 * np.arctan(n / w) / n;
            theta = two_atan_nbyw_by_n * n
                
        tangent = two_atan_nbyw_by_n * q[1:]
        return tangent
                    
    @staticmethod
    def log_quaternion(R):    
        q = SO3.matrix2quaternion(R)
        v = SO3.quaternion2vec(q)
        return v
    
    @staticmethod
    def log_sophus(R):
        return sp.SO3(R).log()
    
    @staticmethod
    def exp(v):
        norm_sq = sum(v**2)
        v_hat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        v_hat_sq = v_hat @ v_hat
        if norm_sq < np.finfo(float).eps * np.finfo(float).eps:
            return np.eye(3) + v_hat + v_hat_sq * 0.5
        else:
            norm = np.sqrt(norm_sq)
            return np.eye(3) + np.sin(norm) / norm * v_hat + (1 - np.cos(norm)) / norm_sq * v_hat_sq

## Test log maps

In [ ]:
np.random.seed(42)


class TestSO3LogMap():
    def __init__(self):
        self.v_own = np.array([[0, -np.pi, 0], [0, np.pi - 1e-8, 0], [0, 0, -np.pi + 1e-7],
                               [np.pi - 1e-6, 0, 0], [-np.pi + 1e-5, 0, 0],
                               [-np.pi/2, 0, 0],
                               [0, 0, 0], [0, 0, -1e-8], [1e-5, 0, 0], [1e-4, -1e-4, 0], 
                               [-1, -1, -1]])
        self.v_generated = self.generate(10000)
        self.v_set = np.vstack((self.v_own, self.v_generated))
        
    def test_log_map(self, log_map):
        correct, total = 0, 0
        for i, v in enumerate(self.v_set):
            sp_obj = sp.SO3.exp(v)
            R = sp_obj.matrix()
            pred = log_map(R)
            if abs(np.linalg.norm(v) - np.pi) < 1e-8:
                if np.any(np.sign(v) * np.sign(pred) < 0):
                    pred = -pred
            try:
                np.testing.assert_array_almost_equal(pred, v, 8)
            except AssertionError as aerr:
                print("Assertion Error in function: ", log_map.__name__, aerr)
                print("Test ", i+1, ", Angle: ", np.linalg.norm(v), "\n")
                total += 1
            else:
                correct += 1
                total += 1
        return correct, total
            
    def run(self):
        correct_log_baseline, total_log_baseline = self.test_log_map(SO3.log_baseline)
        correct_log_pi, total_log_pi = self.test_log_map(SO3.log_pi)
        correct_log_quaternion, total_log_quaternion = self.test_log_map(SO3.log_quaternion)
        correct_sophus, total_sophus = self.test_log_map(SO3.log_sophus)
        
        print(f"SO3.log_baseline() had: {total_log_baseline - correct_log_baseline} / {total_log_baseline} "
              f"failed tests")
        print(f"SO3.log_pi() had: {total_log_pi - correct_log_pi} / {total_log_pi} failed tests")
        print(f"SO3.log_quaternion() had: {total_log_quaternion - correct_log_quaternion} / "
              f"{total_log_quaternion} failed tests")
        print(f"sophus.SO3(R).log() had: {total_sophus - correct_sophus} / {total_sophus} failed tests")
        
    def generate(self, k=1000):
        angles_unit_vecs = np.random.uniform(low=0, high=2 * np.pi, size=(k, 2))
        cos_a = np.cos(angles_unit_vecs)
        sin_a = np.sin(angles_unit_vecs)
        x = cos_a[:, 0] * cos_a[:, 1]
        y = sin_a[:, 0] * cos_a[:, 1]
        z = sin_a[:, 1]
        unit_vecs = np.vstack((x, y, z)).transpose()
        
        thetas = np.random.uniform(low=0, high=np.pi, size=(k-20))
        thetas = np.hstack((thetas, np.array(10*[np.pi])))
        thetas = np.hstack((thetas, np.array(9*[1e-8])))
        thetas = np.hstack((thetas, np.array([0])))
        
        v = thetas[:, None] * unit_vecs
        return v
        

In [ ]:
tests = TestSO3LogMap()
tests.run()

### Conclusion:
1. $\log$ map in __baseline__ implementation failed for the rotation matrices corresponding to rotations by angles close to $\pi$ (1e-8 neighborhood -> fails, 1e-7 neighborhood -> 1e-1 max abs difference, 1e-6 -> 1e-4, 1e-5 -> 1e-7 )
2. $\log$ map implementation adjusted for **angle $\pi$** gives correct results with precision up to 1e-8 max absolute difference. At angle exactly equal to $\pi$ the $\log$ map is defined up to an overall sign.
3. $\log$ map implementation using intermediate **quaternion** representation (matrix -> quaternion -> angle/axis) gives correct results everywhere.
4. **Sophus** implementation of $\log$ map gives correct results too. Internally it uses the "matrix to quaternion" conversion from Eigen library.

## 2. Deriving the Jacobians of  $\log$ map

$$ \frac{\partial log(R)^{\vee}}{\partial R} $$
It is well-defined only in the manifold of $SO(3)$ matrices. 
However, we are trying to approximate it as if it was defined on the whole space of $M(3, 3)$ matrices.

The resulting Jacobian will have shape $3 \times 9$, where 9 corresponds to vectorized rotation matrix in row-major way.

In [ ]:
def num_diff(N, M, func, a, eps=1e-8, project=None):
    J = np.zeros((N, M))
    h = np.zeros(M)
    shape_a = a.shape
    a_flattened = np.reshape(a, -1)
    precomp_func = func(a).reshape(N)
    
    for i in range(M):
        h[i] = eps
        disturbed = np.reshape(a_flattened + h, shape_a)
        if project is not None:
            disturbed = project(disturbed)
        J[:, i] = (func(disturbed).reshape(N) - precomp_func) / (eps)
        h[i] = 0
    return J

# From sophus (Python) documentation
# sp.to_orthogonal(R)
# 3.if R is not a strict rotation matrix, normalize it. Uses Eigen3 
# Eigen::Quaterniond q(R);
# q.normalized().toRotationMatrix();

def makeRotationMatrix(R):
    assert(R.shape == (3, 3))
    
    U, s, Vtr = np.linalg.svd(R, full_matrices=True)
    det = np.linalg.det(U @ Vtr)
    Diag = np.eye(3)
    Diag[2, 2] = det
    return U @ Diag @ Vtr

In [ ]:
def Dx_log_x_baseline(R):
    trR = R[0, 0] + R[1, 1] + R[2, 2]
    cos_theta = max(min(0.5 * (trR - 1), 1), -1)
    if cos_theta > 1 - 1e-8:
        return np.array([[0, 0, 0, 0, 0, -0.5, 0, 0.5, 0],
                       [0, 0, 0.5, 0, 0, 0, -0.5, 0, 0], 
                       [0, -0.5, 0, 0.5, 0, 0, 0, 0, 0]])
    
    theta = np.arccos(cos_theta)
    sin_theta = np.sqrt(1 - cos_theta * cos_theta)
    n = np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], 
                  R[1, 0] - R[0, 1]])
    c = (theta * cos_theta - sin_theta) / (4 * sin_theta**3)
    a = c * n
    b = theta / (2 * sin_theta)
    
    return np.array([[a[0], 0, 0, 0, a[0], -b, 0, b, a[0]],
                    [a[1], 0, b, 0, a[1], 0, -b, 0, a[1]],
                    [a[2], -b, 0, b, a[2], 0, 0, 0, a[2]]])

In [ ]:
def Dx_log_x_pi(R):
    trR = R[0, 0] + R[1, 1] + R[2, 2]
    cos_theta = max(min(0.5 * (trR - 1), 1), -1)
    sin_theta = np.sqrt(1 - cos_theta * cos_theta)
    theta = np.arctan2(sin_theta, cos_theta)
    R_minus_R_T_vee = np.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])

    if abs(3 - trR) < 1e-8:
        # return Jacobian of log map at Theta = 0
        return np.array([[0, 0, 0, 0, 0, -0.5, 0, 0.5, 0],
                           [0, 0, 0.5, 0, 0, 0, -0.5, 0, 0], 
                           [0, -0.5, 0, 0.5, 0, 0, 0, 0, 0]])
    
    S = R + R.transpose() + (1 - trR) * np.eye(3)
    rest_tr = 3 - trR
    n = np.ones(3)
    # Fix modules of n_i
    for i in range(3):
        n[i] = np.sqrt(max(0, S[i, i] / rest_tr))
    max_i = np.argmax(n)

    # Fix signs according to the sign of max element
    for i in range(3):
        if i != max_i:
            n[i] *= np.sign(S[max_i, i])

    # Fix an overall sign
    if any(np.sign(n) * np.sign(R_minus_R_T_vee) < 0):
        n = -n
    
    dn_dR_11 = np.zeros(3)
    dn_dR_22 = np.zeros(3)
    dn_dR_33 = np.zeros(3)
    if abs(n[0]) > np.finfo(float).eps:
        dn_dR_11[0] = (S[0, 0] + rest_tr) / (2 * n[0] * rest_tr * rest_tr)
        dn_dR_22[0] = (S[0, 0] - rest_tr) / (2 * n[0] * rest_tr * rest_tr)
        dn_dR_33[0] = dn_dR_22[0]
    if abs(n[1]) > np.finfo(float).eps:
        dn_dR_11[1] = (S[1, 1] - rest_tr) / (2 * n[1] * rest_tr * rest_tr)
        dn_dR_22[1] = (S[1, 1] + rest_tr) / (2 * n[1] * rest_tr * rest_tr)
        dn_dR_33[1] = dn_dR_11[1]
    if abs(n[2]) > np.finfo(float).eps:
        dn_dR_11[2] = (S[2, 2] - rest_tr) / (2 * n[2] * rest_tr * rest_tr)
        dn_dR_22[2] = dn_dR_11[2]
        dn_dR_33[2] = (S[2, 2] + rest_tr) / (2 * n[2] * rest_tr * rest_tr)
    
    dtheta_d_R_11 = -sin_theta * 0.5 + cos_theta * 0.5 * (dn_dR_11.dot(R_minus_R_T_vee))
    dtheta_d_R_22 = -sin_theta * 0.5 + cos_theta * 0.5 * (dn_dR_22.dot(R_minus_R_T_vee))
    dtheta_d_R_33 = -sin_theta * 0.5 + cos_theta * 0.5 * (dn_dR_33.dot(R_minus_R_T_vee)) 
    
    dtheta_d_R_12 = cos_theta * 0.5 * (-n[2])
    dtheta_d_R_13 = cos_theta * 0.5 * (n[1])
    dtheta_d_R_23 = cos_theta * 0.5 * (-n[0])
    dtheta_d_R_21 = -dtheta_d_R_12
    dtheta_d_R_31 = -dtheta_d_R_13
    dtheta_d_R_32 = -dtheta_d_R_23
    
    J = np.zeros((3, 3, 3))
    
    J[0, 0, 0] = dtheta_d_R_11 * n[0] + theta * dn_dR_11[0]
    J[0, 1, 1] = dtheta_d_R_22 * n[0] + theta * dn_dR_22[0]
    J[0, 2, 2] = dtheta_d_R_33 * n[0] + theta * dn_dR_33[0]
    J[0, 0, 1] = dtheta_d_R_12 * n[0]
    J[0, 0, 2] = dtheta_d_R_13 * n[0]
    J[0, 1, 2] = dtheta_d_R_23 * n[0]
    J[0, 1, 0] = -J[0, 0, 1]
    J[0, 2, 0] = -J[0, 0, 2]
    J[0, 2, 1] = -J[0, 1, 2]
    
    J[1, 0, 0] = dtheta_d_R_11 * n[1] + theta * dn_dR_11[1]
    J[1, 1, 1] = dtheta_d_R_22 * n[1] + theta * dn_dR_22[1]
    J[1, 2, 2] = dtheta_d_R_33 * n[1] + theta * dn_dR_33[1]
    J[1, 0, 1] = dtheta_d_R_12 * n[1]
    J[1, 0, 2] = dtheta_d_R_13 * n[1]
    J[1, 1, 2] = dtheta_d_R_23 * n[1]
    J[1, 1, 0] = -J[1, 0, 1]
    J[1, 2, 0] = -J[1, 0, 2]
    J[1, 2, 1] = -J[1, 1, 2]
    
    J[2, 0, 0] = dtheta_d_R_11 * n[2] + theta * dn_dR_11[2]
    J[2, 1, 1] = dtheta_d_R_22 * n[2] + theta * dn_dR_22[2]
    J[2, 2, 2] = dtheta_d_R_33 * n[2] + theta * dn_dR_33[2]
    J[2, 0, 1] = dtheta_d_R_12 * n[2]
    J[2, 0, 2] = dtheta_d_R_13 * n[2]
    J[2, 1, 2] = dtheta_d_R_23 * n[2]
    J[2, 1, 0] = -J[2, 0, 1]
    J[2, 2, 0] = -J[2, 0, 2]
    J[2, 2, 1] = -J[2, 1, 2]
    
    return J.reshape(3, 9)

In [ ]:
def Dquaternion_DR(R):
    """Computes d quaternion(R) / d R , 4 x 9 Jacobian."""
    J_quat = np.zeros((4, 3, 3))
    t = R[0, 0] + R[1, 1] + R[2, 2]
    if t > 0:
        # case 1
        isqrt_t = 1 / np.sqrt(1 + t)
        J_quat[0, :, :] = 0.25 * isqrt_t * np.eye(3)
        J_quat[1, :, :] = -0.25 * isqrt_t / (1 + t) * (R[2, 1] - R[1, 2]) * np.eye(3)
        J_quat[1, 2, 1] = 0.5 * isqrt_t
        J_quat[1, 1, 2] = -0.5 * isqrt_t
        J_quat[2, :, :] = -0.25 * isqrt_t / (1 + t) * (R[0, 2] - R[2, 0]) * np.eye(3)
        J_quat[2, 0, 2] = 0.5 * isqrt_t
        J_quat[2, 2, 0] = -0.5 * isqrt_t
        J_quat[3, :, :] = -0.25 * isqrt_t / (1 + t) * (R[1, 0] - R[0, 1]) * np.eye(3)
        J_quat[3, 1, 0] = 0.5 * isqrt_t
        J_quat[3, 0, 1] = -0.5 * isqrt_t
    else:
        i = 0
        if R[1, 1] > R[0, 0]:
            i = 1
        if R[2, 2] > R[i, i]:
            i = 2
        j = (i + 1) % 3
        k = (j + 1) % 3
        r = np.sqrt(R[i, i] - R[j, j] - R[k, k] + 1)
        i_r = 1 / r
        i_r_cube = 1 / ((R[i, i] - R[j, j] - R[k, k] + 1) * r)
        r_eye = np.eye(3)
        r_eye[j, j] = -1
        r_eye[k, k] = -1
        J_quat[1 + i, :, :] = 0.25 * i_r * r_eye
        
        J_quat[0, :, :] = -0.25 * (R[k, j] - R[j, k]) * i_r_cube * r_eye
        J_quat[0, k, j] = 0.5 * i_r
        J_quat[0, j, k] = -0.5 * i_r
        J_quat[1+j, :, :] = -0.25 * (R[j, i] + R[i, j]) * i_r_cube * r_eye
        J_quat[1+j, j, i] = 0.5 * i_r
        J_quat[1+j, i, j] = 0.5 * i_r
        J_quat[1+k, :, :] = -0.25 * (R[k, i] + R[i, k]) * i_r_cube * r_eye
        J_quat[1+k, k, i] = 0.5 * i_r
        J_quat[1+k, i, k] = 0.5 * i_r
    return J_quat.reshape((4, 9))


def Dlog_Dquaternion(q):
    """Computes d log(q) / d q , 3 x 4 Jacobian."""
    J_vec = np.zeros((3, 4))
    squared_n = q[1]*q[1] + q[2]*q[2] + q[3]*q[3]
    n = np.sqrt(squared_n)
    w = q[0]
    squared_w = w*w
    if squared_n < np.finfo(float).eps * np.finfo(float).eps:
        # Theta close to 0
        two_atan_nbyw_by_n = 2 / w - 2.0 / 3 * squared_n / (w * squared_w)
        d_q0 = -2 / squared_w + 2 * squared_n / (squared_w * squared_w)
        d_q1 = -4.0 / 3 * q[1] / (w * squared_w)
        d_q2 = -4.0 / 3 * q[2] / (w * squared_w)
        d_q3 = -4.0 / 3 * q[3] / (w * squared_w)
    else:
        if (abs(w) < np.finfo(float).eps):
            # Theta close to pi
            d_q0 = -2 / (squared_w + squared_n)
            if w >= 0:
                # From left
                # 2 * arccos(w), w -> 0
                # arcos'(w) = -1 / (sqrt(1 - w*w)) = |w=0| = -1
                
                two_atan_nbyw_by_n = np.pi / n
                d_q1 = -q[1] * np.pi / (squared_n * n)
                d_q2 = -q[2] * np.pi / (squared_n * n)
                d_q3 = -q[3] * np.pi / (squared_n * n)
            else:
                # From right
                two_atan_nbyw_by_n = -np.pi / n
                d_q1 = q[1] * np.pi / (squared_n * n)
                d_q2 = q[2] * np.pi / (squared_n * n)
                d_q3 = q[3] * np.pi / (squared_n * n)
        else:
            # Regular case
            two_atan_nbyw_by_n = 2 * np.arctan(n / w) / n
            d_q0 = -2 / (squared_w + squared_n)
            c0 = (2 / (squared_n * (w + squared_n/w)) - 2 * np.arctan(n / w) / (squared_n * n))
            d_q1 = c0 * q[1]
            d_q2 = c0 * q[2]
            d_q3 = c0 * q[3]
        
    J_vec[:, 0] = d_q0 * q[1:]
    J_vec[:, 1] = d_q1 * q[1:]
    J_vec[0, 1] += two_atan_nbyw_by_n
    J_vec[:, 2] = d_q2 * q[1:]
    J_vec[1, 2] += two_atan_nbyw_by_n
    J_vec[:, 3] = d_q3 * q[1:]
    J_vec[2, 3] += two_atan_nbyw_by_n
    return J_vec


def Dlog_Dquaternion2(q):
    """Computes d log(q) / d q , 3 x 4 Jacobian."""
    J_vec = np.zeros((3, 4))
    squared_n = q[1]*q[1] + q[2]*q[2] + q[3]*q[3]
    n = np.sqrt(squared_n)
    w = q[0]
    squared_w = w*w
    sign = 1
    if w < 0:
        sign = -1
        w = -w
    if squared_n < np.finfo(float).eps * np.finfo(float).eps:
        # n (~Theta) close to 0
        two_atan_nbyw_by_n = 2 / w - 2.0 / 3 * squared_n / (w * squared_w)
        d_q0 = -2 / squared_w + 2 * squared_n / (squared_w * squared_w)
        d_q1 = -4.0 / 3 * q[1] / (w * squared_w)
        d_q2 = -4.0 / 3 * q[2] / (w * squared_w)
        d_q3 = -4.0 / 3 * q[3] / (w * squared_w)
    else:
        # Regular case
        two_atan_nbyw_by_n = 4 * np.arctan(n / (w + np.sqrt(squared_w + squared_n))) / n
        d_q0 = -2 / (squared_w + squared_n)
        c0 = (2 * w - two_atan_nbyw_by_n) / squared_n
        d_q1 = c0 * q[1]
        d_q2 = c0 * q[2]
        d_q3 = c0 * q[3]
        
    J_vec[:, 0] = sign * d_q0 * q[1:]
    J_vec[:, 1] = d_q1 * q[1:]
    J_vec[0, 1] += two_atan_nbyw_by_n
    J_vec[:, 2] = d_q2 * q[1:]
    J_vec[1, 2] += two_atan_nbyw_by_n
    J_vec[:, 3] = d_q3 * q[1:]
    J_vec[2, 3] += two_atan_nbyw_by_n
    J_vec = J_vec * sign
    return J_vec


def Dx_log_x_quaternion(R):
    """Computes d log(R) / d R , 3 x 9 Jacobian."""
    J_quat = Dquaternion_DR(R)
    q = SO3.matrix2quaternion(R)
    J_vec = Dlog_Dquaternion2(q)
    return J_vec @ J_quat

Check `Dquaternion_DR` and `Dlog_Dquaternion` against numerical differentiation.

In [ ]:
# 1. Check Dquaternion_DR
errors = 0
total = 1000
min_angle = np.pi+1
for i in range(total):
    v = np.random.rand(3) * 2 - 1
    norm = np.random.rand(1) * (np.pi + 1e-8)
    v = v / np.linalg.norm(v) * norm
    sp_obj = sp.SO3.exp(v)
    R = sp_obj.matrix()
    
    numerical = num_diff(4, 9, func=SO3.matrix2quaternion, a=R, eps=1e-8, project=None)
    analytical = Dquaternion_DR(R)
    
    precision = 1e-7
    if np.max(np.linalg.norm(numerical - analytical)) > precision:
        errors += 1
        min_angle = min(min_angle, np.linalg.norm(v))
        print("iter = ", i+1)
        print(v, np.linalg.norm(v))
        print("R:\n", R)
        print("num:\n", np.round(numerical, 4))
        print("analytical:\n", np.round(analytical, 4))
        print("diff:\n", np.linalg.norm(numerical - analytical))
print("\n ___________________________\n num of errors: ", errors, "/", total)
# print("min_angle = ", min_angle)

Conclusion:

1. `Dquaternion_DR` gives the same results as the numerical differentiation (without projection) up to precision `1e-7` for `eps=1e-8` in numerical discretization. For different discretization steps the precision changes (non-monotonously).
2. Both numerical differentiation and analytical Jacobian are continuously changing at 0 and at $\pi$.
3. Numerical differentiation with both matrix `projections` change the result.
4. Without projection the Jaobians are sparse with bunch of zeros corresponding to vector part of quaternion.

In [ ]:
# 2. Check Dlog_Dquaternion

def normalize(q):
    norm = np.linalg.norm(q)
    return q / norm

errors = 0
total = 1000
min_angle = np.pi+1
max_angle = -1

for i in range(total):
    v = np.random.rand(3) * 2 - 1
    norm = np.random.rand(1) * (np.pi + 1e-8)
    v = v / np.linalg.norm(v) * norm
    
    sp_obj = sp.SO3.exp(v)
    R = sp_obj.matrix()
    q = SO3.matrix2quaternion(R)
    
    num = num_diff(3, 4, func=SO3.quaternion2vec, a=q, eps=1e-8, project=normalize)
    J_vec = Dlog_Dquaternion2(q)
    precision = 1e-6
    if np.max(np.linalg.norm(J_vec - num))  > precision:
        errors += 1
        min_angle = min(min_angle, np.linalg.norm(v))
        max_angle = max(max_angle, np.linalg.norm(v))
        print("iter = ", i+1)
        print(v, np.linalg.norm(v))
        print("R:\n", R)

        print("numerical:\n", num)
        print("analytical:\n", J_vec)
        print("diff:\n", np.linalg.norm(J_vec - num))

print("\n ___________________________\n num of errors: ", errors, "/", total)
print("min_angle = ", min_angle)
print("max_angle = ", max_angle)

Conclusion:

1. `Dlog_Dquaternion` gives the same results as the numerical differentiation (with and without projection=`normalize`) up to max total difference = 1e-6 for discretization step = 1e-8. The precision depends on discretization step non-monotonously.
2. However, numerical differentiation (with and without projection) diverges for the angles in proximity of Pi (proximity distance $\approx$ step in numerical discretization). Divergence is due to the change of the `sign` of $q_0 = \cos\big(\frac{\theta}{2}\big)$ in numerical step, and only $\frac{\partial v}{\partial q_0}$ diverges. 
3. The Jacobian `Dlog_Dquaternion` is not continuous around $\pi$, particularly, $\frac{\partial \vec{v}}{\partial q_{1:3}}$ changes its sign whenever the angle crosses $\pi$, i.e. $J(\pi - \epsilon) \neq J(\pi+\epsilon)$.

In [ ]:
def Dx_log_x_numerical(R, func=SO3.log_quaternion, project=sp.to_orthogonal, eps=1e-8):
    return num_diff(N=3, M=9, func=func, a=R, eps=eps, project=project)

In [ ]:
SO3.Dx_log_x_baseline = Dx_log_x_baseline
SO3.Dx_log_x_pi = Dx_log_x_pi
SO3.Dx_log_x_quaternion = Dx_log_x_quaternion
SO3.Dx_log_x_numerical = Dx_log_x_numerical

## Looking at the Jacobians of $\log$ map
$$ \frac{\partial log(R)}{\partial R} $$
1. Compare different analytical implementations of Jacobians with numerical differentiation (both projected to SO(3) and not)

In [ ]:
v_set = [np.array([0, 0, 0]), np.array([1, 1, 1]), np.array([0, 1, -0.7]), 
         np.array([0, 1, -0.7])*1e-7, np.array([0, 1, -0.7])*1e-6, np.array([0, 1, -0.7])*1e-5, 
         np.array([0, 1, -0.7])*1e-4, np.array([0, 1, -0.7])*1e-3, np.array([0, 1, -0.7])*1e-2,
         np.array([0, 1, -0.7])*1e-1,
         np.array([np.pi, 0, 0]), np.array([0, np.pi-1e-8, 0]), np.array([0, 0, -np.pi+1e-7]),]

round_out_dec = 4

for i, v in enumerate(v_set):
    print("_________________________________________")
    print("Test ", i+1)
    sp_obj = sp.SO3.exp(v)
    R = sp_obj.matrix()
    print("v = ", v)
    print("Angle = ", np.linalg.norm(v))
    print("R:\n", R)
    
    print("\nAnalytical baseline:")
    print(np.round(SO3.Dx_log_x_baseline(R), round_out_dec))
    print("Numerical baseline raw:")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_baseline, None), round_out_dec))
    print("Numerical baseline projected (norm quaternion):")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_baseline), round_out_dec))
    print("Numerical baseline projected (SVD):")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_baseline, makeRotationMatrix), round_out_dec))
    
    print("\nAnalytical Pi:")
    print(np.round(SO3.Dx_log_x_pi(R), round_out_dec))
    print("Numerical Pi raw:")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_pi, None), round_out_dec))
    print("Numerical Pi projected (norm quaternion):")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_pi), round_out_dec))
    print("Numerical Pi projected (SVD):")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_pi, makeRotationMatrix), round_out_dec))
    
    print("\nAnalytical quaternion:")
    print(np.round(SO3.Dx_log_x_quaternion(R), round_out_dec))
    print("Numerical quaternion raw:")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_quaternion, None), round_out_dec))
    print("Numerical quaternion projected (norm quaternion):")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_quaternion), round_out_dec))
    print("Numerical quaternion projected (SVD):")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_quaternion, makeRotationMatrix), round_out_dec))
    
    print("Numerical Sophus projected (norm quaternion):")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_sophus), round_out_dec))
    print("Numerical Sophus projected (SVD):")
    print(np.round(SO3.Dx_log_x_numerical(R, SO3.log_sophus, makeRotationMatrix), round_out_dec))
    

### Conclusions:
1. Intermediate angles:
    1. All of the Jacobians give different results.
    2. For all methods analytical Jacobian corresponds to numerical raw (except `Analytical Pi` $\neq$ `Numerical Pi raw`).
    3. `Numerical projected (norm quaternion)` $\approx$ `Analytical quaternion`, and the results are the same for all mappings (except null components of mapping around pi). 
    4. `Numerical projected (SVD)` are the same for all mappings (except null components of mapping around pi).
    
2. Small angles (Both `Quaternion` and `Baseline` methods are the best):
    1. `Pi` analytical, numerical diverge starting from angles > 1e-4. (Before 1e-4 we make case differentiation).
    2. `Quaternion` and `Baseline` methods (both analytical and numerical) do not diverge, and are changing continuously with increasing angle.
    3. `Numerical projected (norm quaternion)` prevents `Around Pi` method from diverging.
    4. `Numerical projected (SVD)` does not always prevent `Around Pi` method from diverging.
    
3. Angles close to $\pi$ (`Quaternion` method is best):
    1. `Baseline` analytical, numerical (raw, projected) - diverge.
    2. `Pi` analytical - do not diverge. But it nullifies the components corresponding to n(i) = 0.
    3. `Pi` numerical raw - diverges.
    4. `Quaternion` analytical - does not diverge, and continuously changes with decreasing angle. 
    5. `Quaternion` numerical - diverges at close (~discretization step) proximity of $\pi$ , when the angle crosses $\pi$. 
    6. All `numerical projected` help to avoid divergence. But they also diverge at close (~discretization step) proximity of $\pi$ , when the angle crosses $\pi$.

## 3. Checking the correctness of the above Jacobians.

We already know the expression for $\frac{\partial \log (R \boxplus x)}{\partial x} \big|_{x=0} = J_r^{-1}$, and for $\frac{\partial (R \boxplus x)}{\partial x} \big|_{x=0}$.

So we can check the following chain rule equation:

$$ \frac{\partial \log (R \boxplus x)}{\partial x} \big|_{x=0} = 
\frac{\partial log(R)}{\partial R} 
\frac{\partial (R \boxplus x)}{\partial x} \big|_{x=0}$$

In [ ]:
def log_R_boxplus_x(R, x):
    return SO3.log_sophus(R @ sp.SO3.exp(x).matrix())

def box_plus(R, x):
    return R @ sp.SO3.exp(x).matrix()

def num_diff_2_args(N, M, func, R, eps=1e-8):
    J = np.zeros((N, M))
    h = np.zeros(M)
    precomp_func = func(R, h).reshape(N)
    for i in range(M):
        h[i] = eps
        J[:, i] = (func(R, h).reshape(N) - precomp_func) / (eps)
        h[i] = 0
    return J

In [ ]:
def dx_log_R_mul_exp_x_at_0(R):
    """Computes d log(R exp(x)) / dx at x=0, 3x3 Inverse right Jacobian $J_r^{-1}$"""
    phi = SO3.log_sophus(R)
    phi_norm2 = sum(phi**2)
    phi_hat = sp.SO3.hat(phi)
    phi_hat2 = phi_hat @ phi_hat
    
    J = np.eye(3)
    J += phi_hat / 2;
    if phi_norm2 > np.finfo(float).eps:
        phi_norm = np.sqrt(phi_norm2)
        # assert(phi_norm <= np.pi + np.finfo(float).eps)
        if phi_norm < np.pi - np.finfo(float).eps:
            J += phi_hat2 * (1 / phi_norm2 - (1 + np.cos(phi_norm)) / (2 * phi_norm * np.sin(phi_norm)))
        else:
            # Approximate around pi
            J += phi_hat2 / (np.pi * np.pi)
    else:
        # Around 0
        J += phi_hat2 / 12
    return J

In [ ]:
# 1. Check `dx_log_R_mul_exp_x_at_0` numerically:
errors = 0
total = 1000
min_angle = np.pi+1
max_angle = -1
np.random.seed(42)

for i in range(total):
    v = np.random.rand(3) * 2 - 1
    if i == 0:
        norm = np.pi - 1e-9
    else:
        norm = np.random.rand(1) * (np.pi + 1e-8)
    v = v / np.linalg.norm(v) * norm
    
    sp_obj = sp.SO3.exp(v)
    R = sp_obj.matrix()
    numerical = num_diff_2_args(3, 3, log_R_boxplus_x, R, eps=1e-8)
    analytical = dx_log_R_mul_exp_x_at_0(R)
    
    precision = 1e-6
    if np.max(np.linalg.norm(numerical - analytical)) > precision:
        errors += 1
        min_angle = min(min_angle, np.linalg.norm(v))
        max_angle = max(max_angle, np.linalg.norm(v))
        print("iter = ", i+1)
        print(v, np.linalg.norm(v))
        print("R:\n", R)
        print("num:\n", numerical)
        print("analytical:\n", analytical)
        print("diff:\n", np.linalg.norm(numerical - analytical))
print("\n ___________________________\n num of errors: ", errors, "/", total)
print("min_angle = ", min_angle)
print("max_angle = ", max_angle)


Conclusion:
Comparing analytical and numerical Jacobians at point $x=0$:
$$ \frac{\partial \log (R \boxplus x)}{\partial x} \big|_{x=0} $$
1. Analytical derrivative continuously changes up to $\pi$.
2. However, Analytical Jacobian transposes once the angle crosses $\pi$, i.e. $J(\pi) \approx J(\pi - \epsilon) = J^T(\pi+\epsilon)$.
2. Numerical differentiation diverges close to $\pi$ (log sophus). For close proximity of $\pi$ ~ discretization step.
3. Everywhere else the two methods give the same results up to precision 1e-6.

In [ ]:
def dx_R_mul_exp_x_at_0(R):
    """Computes d (R exp(x)) / dx at x=0, 9x3 Jacobian."""
    G_1 = (R @ sp.SO3.hat([1, 0, 0])).reshape(9)
    G_2 = (R @ sp.SO3.hat([0, 1, 0])).reshape(9)
    G_3 = (R @ sp.SO3.hat([0, 0, 1])).reshape(9)
    J = np.zeros((9, 3))
    J[:, 0] = G_1
    J[:, 1] = G_2
    J[:, 2] = G_3
    return J

In [ ]:
def dx_exp_x(x):
    """Computes d (exp(x)) / dx, 9x3 Jacobian."""
    norm_x = np.linalg.norm(x)
    norm_sq = norm_x * norm_x
    e_1 = [1, 0, 0]
    e_2 = [0, 1, 0]
    e_3 = [0, 0, 1]
    if norm_x < 1e-8:
        G_1 = (sp.SO3.hat(e_1)).reshape(9)
        G_2 = (sp.SO3.hat(e_2)).reshape(9)
        G_3 = (sp.SO3.hat(e_3)).reshape(9)
    else:
        exp_x = sp.SO3.exp(x).matrix()
        x_hat = sp.SO3.hat(x)
        id_minus_exp = np.eye(3) - exp_x
        a_1 = sp.SO3.hat(np.cross(x, id_minus_exp @ e_1))
        a_2 = sp.SO3.hat(np.cross(x, id_minus_exp @ e_2))
        a_3 = sp.SO3.hat(np.cross(x, id_minus_exp @ e_3))
        G_1 = ((x[0]*x_hat + a_1) / norm_sq @ exp_x).reshape(9)
        G_2 = ((x[1]*x_hat + a_2) / norm_sq @ exp_x).reshape(9)
        G_3 = ((x[2]*x_hat + a_3) / norm_sq @ exp_x).reshape(9)
    J = np.zeros((9, 3))
    J[:, 0] = G_1
    J[:, 1] = G_2
    J[:, 2] = G_3
    return J

In [ ]:
def dx_R_mul_exp_x(R, x):
    J = dx_exp_x(x)
    for i in range(3):
        J[:, i] = (R @ (J[:, i]).reshape(3, 3)).reshape(9)
    return J

In [ ]:
# 2. Check `dx_exp_x` numerically:

errors = 0
total = 1000
min_angle = np.pi+1
max_angle = -1
np.random.seed(42)
for i in range(total):
    v = np.random.rand(3) * 2 - 1
    if i == 0:
        norm = np.pi
    else:
        norm = np.random.rand(1) * (np.pi + 1e-8)
    v = v / np.linalg.norm(v) * norm
    numerical = num_diff(9, 3, lambda w: sp.SO3.exp(w).matrix(), v)
    analytical = dx_exp_x(v)
    if np.max(np.linalg.norm(numerical - analytical)) > 1e-6:
        errors += 1
        min_angle = min(min_angle, np.linalg.norm(v))
        max_angle = max(max_angle, np.linalg.norm(v))
        print("iter = ", i+1)
        print(v, np.linalg.norm(v))
        print("R:\n", R)
        print("num:\n", numerical)
        print("analytical:\n", analytical)
        print("diff:\n", np.linalg.norm(numerical - analytical))
print("\n ___________________________\n num of errors: ", errors, "/", total)
print("min_angle = ", min_angle)
print("max_angle = ", max_angle)

In [ ]:
# 3. Check `dx_R_mul_exp_x_at_0` numerically:

errors = 0
total = 1000
min_angle = np.pi+1
max_angle = -1
np.random.seed(42)
for i in range(total):
    v = np.random.rand(3) * 2 - 1
    if i == 0:
        norm = np.pi
    else:
        norm = np.random.rand(1) * (np.pi + 1e-8)
    v = v / np.linalg.norm(v) * norm
    
    sp_obj = sp.SO3.exp(v)
    R = sp_obj.matrix()
    
    numerical = num_diff_2_args(9, 3, box_plus, R)
    analytical = dx_R_mul_exp_x_at_0(R)
    if np.max(np.linalg.norm(numerical - analytical)) > 1e-7:
        errors += 1
        min_angle = min(min_angle, np.linalg.norm(v))
        max_angle = max(max_angle, np.linalg.norm(v))
        print("iter = ", i+1)
        print(v, np.linalg.norm(v))
        print("R:\n", R)
        print("num:\n", numerical)
        print("analytical:\n", analytical)
        print("diff:\n", np.linalg.norm(numerical - analytical))
print("\n ___________________________\n num of errors: ", errors, "/", total)
print("min_angle = ", min_angle)
print("max_angle = ", max_angle)

Conclusion: analytical and numerical jacobians for boxplus operation at $x=0$:
$$\frac{\partial (R \boxplus x)}{\partial x} \big|_{x=0}$$
The two methods give the same results up to precision 1e-7.

### Checking the Jacobians of log map using chain rule:
$$ \frac{\partial \log (R \boxplus x)}{\partial x} \big|_{x=0} = 
\frac{\partial log(y)}{\partial y} \big|_{y=R} 
\frac{\partial (R \boxplus x)}{\partial x} \big|_{x=0}$$

In [ ]:
def test_jacobian_log_map(jacobian_log_map, v, precision=8):
    R = sp.SO3.exp(v).matrix()
    J_boxplus_x_at_0 = dx_R_mul_exp_x_at_0(R)
    J_log_boxplus_x_at_0 = dx_log_R_mul_exp_x_at_0(R)
    J_log_at_R = jacobian_log_map(R)
    predicted = J_log_at_R @ J_boxplus_x_at_0
    try:
        np.testing.assert_array_almost_equal(J_log_boxplus_x_at_0, predicted, precision)
    except AssertionError as aerr:
#         print("Assertion Error in function: ", jacobian_log_map.__name__, aerr)
#         print("v = ", v, ", Angle: ", np.linalg.norm(v), "\n")
        return 1
    return 0

In [ ]:
v_set = [np.array([np.pi, 0, 0]), np.array([0, np.pi-1e-8, 0]), np.array([0, 0, np.pi+1e-7]), 
         np.array([0, 0, 0]), np.array([1, 1, 1]), np.array([0, 1, -0.7])]

errors_baseline, errors_pi, errors_quaternion = 0, 0, 0
errors_num_baseline, errors_num_SVD_baseline, errors_num_sp_baseline = 0, 0, 0
errors_num_pi, errors_num_SVD_pi, errors_num_sp_pi = 0, 0, 0
errors_num_quaternion, errors_num_SVD_quaternion, errors_num_sp_quaternion = 0, 0 ,0

total = 1000
min_angle = np.pi+1
max_angle = -1
np.random.seed(42)

for i in range(total):
    if i < len(v_set):
        v = v_set[i]
    else:
        v = np.random.rand(3) * 2 - 1
        norm = np.random.rand(1) * (np.pi + 1e-8)
        v = v / np.linalg.norm(v) * norm
    errors_baseline += test_jacobian_log_map(SO3.Dx_log_x_baseline, v)
    errors_pi += test_jacobian_log_map(SO3.Dx_log_x_pi, v)
    errors_quaternion += test_jacobian_log_map(SO3.Dx_log_x_quaternion, v)
    
    errors_num_baseline += test_jacobian_log_map(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_baseline, project=None), v)
    errors_num_SVD_baseline += test_jacobian_log_map(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_baseline, project=makeRotationMatrix), v)
    errors_num_sp_baseline += test_jacobian_log_map(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_baseline, project=sp.to_orthogonal), v)
    errors_num_pi += test_jacobian_log_map(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_pi, project=None), v)
    errors_num_SVD_pi += test_jacobian_log_map(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_pi, project=makeRotationMatrix), v)
    errors_num_sp_pi += test_jacobian_log_map(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_pi, project=sp.to_orthogonal), v)
    errors_num_quaternion += test_jacobian_log_map(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_quaternion, project=None), v)
    errors_num_SVD_quaternion += test_jacobian_log_map(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_quaternion, project=makeRotationMatrix), v)
    errors_num_sp_quaternion += test_jacobian_log_map(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_quaternion, project=sp.to_orthogonal), v)


print("___________________________\n errors baseline: ", errors_baseline, "/", total)
print("errors pi: ", errors_pi, "/", total)
print("errors quaternion: ", errors_quaternion, "/", total)
print("errors num raw baseline: ", errors_num_baseline, "/", total)
print("errors num SVD baseline: ", errors_num_SVD_baseline, "/", total)
print("errors num sp baseline: ", errors_num_sp_baseline, "/", total)
print("errors num raw pi: ", errors_num_pi, "/", total)
print("errors num SVD pi: ", errors_num_SVD_pi, "/", total)
print("errors num sp pi: ", errors_num_sp_pi, "/", total)
print("errors num raw quaternion: ", errors_num_quaternion, "/", total)
print("errors num SVD quaternion: ", errors_num_SVD_quaternion, "/", total)
print("errors num sp quaternion: ", errors_num_sp_quaternion, "/", total)
    

### Conclusion:

1. `Dx_log_x_quaternion` gives correct results everywhere (with high precision, < 1e-9 max abs diff).
2. `Dx_log_x_pi` gives inaccurate results for angles close to 0. Also it nullifies rows corresponding to exact zero components of vector $n(i)=0$.
3. `Dx_log_x_baseline` diverges at angle $\pi$, but also gives inaccurate results close to $\pi$ (already in 1e-2 proximity the inaccuracy is up to 1e-6)
4. Interestingly, the Jacobians of $\log$ map derived from different methods give different results. However, after matrix multiplication with jacobian of boxplus at zero, the results become similar or the same.
4. Numerical solutions suffer from discretization-dependent inaccuracies (up 1e-5). They also tend to diverge at angles close to $\pi$.
5. Projections reduce accuracy, but sometimes help to cope with divergence. 



## 4. Use of Jacobians in Taylor expansion

By Taylor expansion, for $x \in \mathbb{R}^3$, $||x|| \rightarrow 0$:

$$ \log (R \boxplus x) = \log(R) + \frac{d log(R)}{dR} \frac{d(R \boxplus x)}{dx} \big|_{x=0} \cdot {x} + O(||x||^2)$$

In [ ]:
def test_taylor_expansion(jacobian_log_map, R, x, precision=6):
    J_boxplus_x_at_0 = dx_R_mul_exp_x_at_0(R)
    J_log_boxplus_x_at_0 = dx_log_R_mul_exp_x_at_0(R)
    J_log_at_R = jacobian_log_map(R)
    logR = SO3.log_quaternion(R)
    logR_plus_x = log_R_boxplus_x(R, x)
    predicted = logR + J_log_at_R @ J_boxplus_x_at_0 @ x
    err = np.linalg.norm(predicted - logR_plus_x)
    try:
        np.testing.assert_array_almost_equal(logR_plus_x, predicted, precision)
    except AssertionError as aerr:
#         print("Assertion Error in function: ", jacobian_log_map.__name__, aerr)
#         print("v = ", v, ", Angle: ", np.linalg.norm(v), "\n")
        return 1
    return 0

In [ ]:
v_set = [np.array([0.5, 1, -0.7]), np.array([np.pi, 0, 0]), np.array([0, np.pi-1e-8, 0]), 
         np.array([0, 0, np.pi+1e-7]), 
         np.array([0, 0, 0]), np.array([1, 1, 1]), 
         np.array([0.5, 1, -0.7]) * 1e-8, np.array([0.5, 1, -0.7]) * 1e-7,
        np.array([1, 1, 1]) * 1e-6, np.array([1, 1, 1])*1e-5, np.array([1, 1, 1])*1e-4,
        np.array([1, 1, 1])*1e-3,np.array([1, 1, 1])*1e-2, np.array([1, 1, 1])*1e-1]

errors_baseline, errors_pi, errors_quaternion = 0, 0, 0
errors_num_baseline, errors_num_SVD_baseline, errors_num_sp_baseline = 0, 0, 0
errors_num_pi, errors_num_SVD_pi, errors_num_sp_pi = 0, 0, 0
errors_num_quaternion, errors_num_SVD_quaternion, errors_num_sp_quaternion = 0, 0 ,0

total = 1000
min_angle = np.pi+1
max_angle = -1
np.random.seed(42)
precision = 8

norm_x = 1e-3

for i in range(total):
    if i < len(v_set):
        v = v_set[i]
    else:
        v = np.random.rand(3) * 2 - 1
        norm = np.random.rand(1) * (np.pi + 1e-8)
        v = v / np.linalg.norm(v) * norm
    R = sp.SO3.exp(v).matrix()
    x = np.random.rand(3) * 2 - 1
    x = x / np.linalg.norm(x) * norm_x
    
    errors_baseline += test_taylor_expansion(SO3.Dx_log_x_baseline, R, x)
    errors_pi += test_taylor_expansion(SO3.Dx_log_x_pi, R, x)
    errors_quaternion += test_taylor_expansion(SO3.Dx_log_x_quaternion, R, x)
    
    errors_num_baseline += test_taylor_expansion(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_baseline, project=None), R, x)
    errors_num_SVD_baseline += test_taylor_expansion(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_baseline, project=makeRotationMatrix), R, x)
    errors_num_sp_baseline += test_taylor_expansion(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_baseline, project=sp.to_orthogonal), R, x)
    errors_num_pi += test_taylor_expansion(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_pi, project=None), R, x)
    errors_num_SVD_pi += test_taylor_expansion(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_pi, project=makeRotationMatrix), R, x)
    errors_num_sp_pi += test_taylor_expansion(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_pi, project=sp.to_orthogonal), R, x)
    errors_num_quaternion += test_taylor_expansion(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_quaternion, project=None), R, x)
    errors_num_SVD_quaternion += test_taylor_expansion(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_quaternion, project=makeRotationMatrix), R, x)
    errors_num_sp_quaternion += test_taylor_expansion(
        lambda R: SO3.Dx_log_x_numerical(R, func=SO3.log_quaternion, project=sp.to_orthogonal), R, x)


print("___________________________\n errors baseline: ", errors_baseline, "/", total)
print("errors pi: ", errors_pi, "/", total)
print("errors quaternion: ", errors_quaternion, "/", total)
print("errors num raw baseline: ", errors_num_baseline, "/", total)
print("errors num SVD baseline: ", errors_num_SVD_baseline, "/", total)
print("errors num sp baseline: ", errors_num_sp_baseline, "/", total)
print("errors num raw pi: ", errors_num_pi, "/", total)
print("errors num SVD pi: ", errors_num_SVD_pi, "/", total)
print("errors num sp pi: ", errors_num_sp_pi, "/", total)
print("errors num raw quaternion: ", errors_num_quaternion, "/", total)
print("errors num SVD quaternion: ", errors_num_SVD_quaternion, "/", total)
print("errors num sp quaternion: ", errors_num_sp_quaternion, "/", total)

Conclusion:

1. The approximation up to precision $||x||^2$ (||x|| < 1e-1) works for all vectors, except the case when $\log(R \boxplus x)$ changes the sign of $\log(R)$

References:

* Lie theory, differential calculus on manifolds:

    [[1] A micro Lie theory for state estimation in robotics. Joan Sola, Jeremie Deray, Dinesh Atchuthan, 2020](https://arxiv.org/pdf/1812.01537.pdf)

    [[2] A  Primer  on  the  Differential  Calculus  of  3D  Orientations. Michael Bloesch et al., 2016](https://arxiv.org/pdf/1606.05285.pdf)
    
    [[3] A compact formula for the derivative of a 3-D rotation in exponential coordinates. Guillermo Gallego, Anthony Yezzi, 2014](https://arxiv.org/pdf/1312.0788.pdf)

## Use case: Ceres optimization

In [ ]:
pyceres_location="/home/nurlanov/tum/lib/ceres-bin/lib/"

import sys
sys.path.insert(0, pyceres_location)

In [ ]:
import PyCeres

In [ ]:
from jax import grad, jacfwd
import jax.numpy as jnp
import math

In [ ]:
def jax_matrix2quaternion(R):    
    # quaternion [q_0, q_{1:3}] = [c, vec]
    q = jnp.zeros(4)
    t = R[0, 0] + R[1, 1] + R[2, 2]
    if t > 0:
        # case 1
        t = jnp.sqrt(1 + t)
        q[0] = 0.5 * t
        t = 0.5 / t
        q[1] = (R[2, 1] - R[1, 2]) * t
        q[2] = (R[0, 2] - R[2, 0]) * t
        q[3] = (R[1, 0] - R[0, 1]) * t
    else:
        i = 0
        if R[1, 1] > R[0, 0]:
            i = 1
        if R[2, 2] > R[i, i]:
            i = 2
        j = (i + 1) % 3
        k = (j + 1) % 3
        t = jnp.sqrt(R[i, i] - R[j, j] - R[k, k] + 1)
        q[1+i] = 0.5 * t
        t = 0.5 / t
        q[0] = (R[k, j] - R[j, k]) * t
        q[1+j] = (R[j, i] + R[i, j]) * t
        q[1+k] = (R[k, i] + R[i, k]) * t
    return q
    
def jax_quaternion2vec(q):
    squared_n = q[1]*q[1] + q[2]*q[2] + q[3]*q[3]
    w = q[0]
    theta = 0
    if squared_n < np.finfo(float).eps * np.finfo(float).eps:
        squared_w = w*w
        two_atan_nbyw_by_n = 2 / w - 2.0 / 3 * squared_n / (w * squared_w) 
        theta = 2 * squared_n / w
    else:
        n = jnp.sqrt(squared_n)
        if (abs(w) < np.finfo(float).eps):
            if w >= 0:
                two_atan_nbyw_by_n = np.pi / n
            else:
                two_atan_nbyw_by_n = -np.pi / n
        else:
            two_atan_nbyw_by_n = 2 * jnp.arctan(n / w) / n;
        theta = two_atan_nbyw_by_n * n

    tangent = two_atan_nbyw_by_n * q[1:]
    return tangent
               
def jax_log_quaternion(R):    
    q = jax_matrix2quaternion(R)
    v = jax_quaternion2vec(q)
    return v
    
def jax_exp(v):
    norm_sq = sum(v**2)
    v_hat = jnp.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    v_hat_sq = v_hat @ v_hat
    if norm_sq < np.finfo(float).eps * np.finfo(float).eps:
        return np.eye(3) + v_hat + v_hat_sq * 0.5
    else:
        norm = jnp.sqrt(norm_sq)
        return np.eye(3) + jnp.sin(norm) / norm * v_hat + (1 - jnp.cos(norm)) / norm_sq * v_hat_sq
    
def jax_log_baseline(R):    
    trR = R[0, 0] + R[1, 1] + R[2, 2]
    cos_theta = max(min(0.5 * (trR - 1), 1), -1)
    sin_theta = 0.5 * jnp.sqrt(max(0, (3 - trR) * (1 + trR)))
    theta = jnp.arctan2(sin_theta, cos_theta)
    R_minus_R_T_vee = jnp.array([R[2, 1] - R[1, 2], R[0, 2] - R[2, 0], R[1, 0] - R[0, 1]])

    if abs(3 - trR) < 1e-8:
        # return log map at Theta = 0
        c = 0.5 * (1 + theta*theta / 6 + 7 / 360 * (theta**4))
        return c * R_minus_R_T_vee

    # it diverges around theta=pi
    v = theta / (2 * sin_theta) * R_minus_R_T_vee
    return v

In [ ]:
def jax_residual_calc(R_inv_mat, x):
    x_mat = jax_exp(x)
    product = R_inv_mat @ x_mat
    residual = jax_log_baseline(product)
    
#     x_mat = sp.SO3.exp(x).matrix()
#     product = R_inv_mat @ x_mat
#     residual = sp.SO3.log(product)
    return residual

def residual_calc(R_inv_mat, x):
    x_mat = SO3.exp(x)
    product = R_inv_mat @ x_mat
    residual = SO3.log_quaternion(product)
    return residual

def jacobian_calc(R_inv_mat, x):
    x_mat = SO3.exp(x)
    product = R_inv_mat @ x_mat
    jacobian = SO3.Dx_log_x_quaternion(product) @ dx_R_mul_exp_x(R_inv_mat, x)
    return jacobian


class BoxMinusAutoDiff(PyCeres.CostFunction):
    def __init__(self, R_inv):
        super().__init__()
        self.set_num_residuals(3)
        self.set_parameter_block_sizes([3])
        self.R_inv = R_inv

    def Evaluate(self, parameters, residuals, jacobians):
        x = parameters[0]
        t = residual_calc(self.R_inv, x)
        for i in range(len(residuals)):
            residuals[i] = t[i]

        if (jacobians!=None):
            J = jacfwd(lambda x: jax_residual_calc(self.R_inv, x))(x)
            print("J auto:\n", J)
            J_a = jacobian_calc(self.R_inv, x)
            print("J_analytical:\n", J_a)
            tt = J_a.reshape(-1, order="C")
#             tt = J.reshape(-1, order="C")
            for i in range(len(jacobians[0])):
                jacobians[0][i] = tt[i]
#             jacobians[0] = grad(lambda x: residual_calc(self.R_inv, x))(x)

        return True

In [ ]:
# The variable to solve for with its initial value.
initial_x = [0.1, 0.05, -0.7]
x = np.array(initial_x) # Requires the variable to be in a numpy array
v = np.random.rand(3) * 2 - 1
v = v / np.linalg.norm(v) * (np.pi)
R = sp.SO3.exp(x) * sp.SO3.exp(v)
R_inv_mat = R.inverse().matrix()
R_mat = R.matrix()

# Here we create the problem as in normal Ceres
problem=PyCeres.Problem()

cost_function=BoxMinusAutoDiff(R_inv_mat)

# Add the costfunction and the parameter numpy array to the problem
problem.AddResidualBlock(cost_function,None,x) 

# Setup the solver options as in normal ceres
options=PyCeres.SolverOptions()

# Ceres enums live in PyCeres and require the enum Type
options.linear_solver_type=PyCeres.LinearSolverType.DENSE_QR
options.minimizer_progress_to_stdout=True
summary=PyCeres.Summary()

# Solve as you would normally
PyCeres.Solve(options,problem,summary)
print(summary.BriefReport() + " \n")
print( "x : ", initial_x, " -> ", x, "\n")

print("Target: ", SO3.log_quaternion(R.matrix()))
print("Optimized: ", x)
norm_x = np.linalg.norm(x)
print("norm optimized = ", norm_x)
proj_x = SO3.log_quaternion(SO3.exp(x))
print("Optimized projected back: ", proj_x)

err = np.linalg.norm(proj_x - SO3.log_quaternion(R.matrix()))
new_x = x / norm_x * (norm_x - 2*np.pi)

print("Optimized energy: ", SO3.log_quaternion(R_inv_mat @ SO3.exp(x)))
print("Difference: ", err)